In [ ]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [7]:
import optuna
import sqlite3

# Enable WAL mode
conn = sqlite3.connect("optuna_hyperparam_tuning.db")
conn.execute("PRAGMA journal_mode=WAL;")
conn.close()
print ("done")


done


In [1]:
import optuna

storage_url = "sqlite:////itet-stor/trachsele/net_scratch/tl4rec/optuna_hyperparam_tuning.db"
#optuna.delete_study(study_name="my_hyperparam_study", storage=storage_url)
#print ("deleted")
# Get all study names
study_names = optuna.get_all_study_names(storage=storage_url)
print("Existing studies:", study_names)

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Existing studies: ['my_hyperparam_study']


In [1]:
%run script/pretrain.py -c config/recommender/pretrain_all.yaml --gpus [0]

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
19:28:23   Random seed: 1024
19:28:23   Config file: config/recommender/pretrain_all.yaml
19:28:23   {'dataset': {'class': 'JointDataset',
             'graphs': ['Amazon_Beauty'],
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.05,
                                                 'hidden_dims': [4, 4, 4],
                                                 'use_dropout': True,
                                                 'use_layer_norm': True},
                              'embedding_item': {'dropout_rate': 0.1,
                                                 'hidden_dims': [16, 16],
              

We will evaluate vs 100 negatives
discard node_features


19:28:27   Fast evaluation on 500 samples in validation
19:28:27   ------------------------------
19:28:27   Number of parameters: 0
19:28:27   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:28:27   Epoch 0 begin


Load rspmm extension. This may take a while...


19:28:29   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:28:29   binary cross entropy: 0.70039
19:28:49   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:28:49   Epoch 0 end
19:28:49   ------------------------------
19:28:49   average binary cross entropy: 0.649191
19:28:49   Save checkpoint to model_epoch_1.pth
19:28:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:28:50   Evaluate on valid


UnboundLocalError: local variable 'tail_ndcgs' referenced before assignment

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset LastFM --epochs 1 --bpe 10000 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_4_40_proj.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
19:42:47   Random seed: 1024
19:42:47   Config file: config/recommender/notebook_cfg.yaml
19:42:47   {'checkpoint': '/itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_4_40_proj.pth',
 'dataset': {'class': 'LastFM',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.2,
                                                 'hidden_dims': [4, 4, 4],
                                                 'use_dropout': True,
                                                 'use_layer_norm': True},
                              'embedding_item': {'dropout_rate': 0.1,
                                              

We will evaluate vs 1000 negatives
bpe 9930.0
discarded user_features
edge_attr.shape = torch.Size([79442, 1])
freeze my backbone
unfreeze my backbone
Load rspmm extension. This may take a while...


19:42:48   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:42:48   binary cross entropy: 0.691503
19:42:48   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:42:48   Epoch 0 end
19:42:48   ------------------------------
19:42:48   average loss: 1.10721
19:42:48   Save checkpoint to model_epoch_1.pth
19:42:48   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:42:48   Evaluate on valid
19:43:10   mr: 23.6892
19:43:10   mrr: 0.24388
19:43:10   hits@1: 0.143246
19:43:10   hits@3: 0.252836
19:43:10   hits@10: 0.455705
19:43:10   ndcg@20: 0.31544
19:43:10   Load checkpoint from model_epoch_1.pth
19:43:10   Save final_ckpt to /itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain/LastFM.pth
19:43:10   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:43:10   Evaluate on valid
19:43:40   mr: 233.527
19:43:40   mrr: 0.0966322
19:43:40   hits@1: 0.0599044
19:43:40   hits@3: 0.0959032
19:43:40   hits@10: 0.153558
19:43:40   ndcg@20: 0.202764
19:43:40   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
19:43:40   Evaluate on test


In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 2 --bpe 20000 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_4_40_proj.pth

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Epinions --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
15:26:37   Random seed: 1024
15:26:37   Config file: config/recommender/notebook_cfg.yaml
15:26:37   {'checkpoint': None,
 'dataset': {'class': 'Ml1m',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.2,
                                                 'hidden_dims': [4, 4, 4],
                                                 'use_dropout': True,
                                                 'use_layer_norm': True},
                              'embedding_item': {'dropout_rate': 0.1,
                                                 'hidden_dims': [16, 16],
                                            

Hi we are in the process_df method.
This is the shape of the original df (798832, 4)
This is the shape after we dropped cols (798832, 2)
Number of null rows: 0
Feature tensor shape(before constant_col_detection): torch.Size([798832, 2])
final Feature tensor shape: torch.Size([798832, 2])
process_df was sucessful
Hi we are in the process_df method.
This is the shape of the original df (199707, 4)
This is the shape after we dropped cols (199707, 2)
Number of null rows: 0
Feature tensor shape(before constant_col_detection): torch.Size([199707, 2])
final Feature tensor shape: torch.Size([199707, 2])
process_df was sucessful
Graph validation passed!
Graph validation passed!
Graph validation passed!
Graph looks good!


Done!
15:26:47   Ml1m dataset
15:26:47   #train: 637734, #valid: 161098, #test: 199707


discarded user_features
edge_attr.shape = torch.Size([1275468, 2])


15:26:47   ------------------------------
15:26:47   Number of parameters: 53945
15:26:47   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:26:47   Epoch 0 begin


freeze my backbone
unfreeze my backbone
Load rspmm extension. This may take a while...


15:26:49   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:26:49   binary cross entropy: 0.694133
15:26:49   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:26:49   Epoch 0 end
15:26:49   ------------------------------
15:26:49   average loss: 1.53648
15:26:49   Save checkpoint to model_epoch_1.pth
15:26:50   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:26:50   Evaluate on valid


In [ ]:
import torch
from rspmm import generalized_rspmm

# Test input data
edge_index = torch.tensor([[0, 1, 2], [1, 2, 3]], dtype=torch.long)
edge_type = torch.tensor([0, 1, 2], dtype=torch.long)
edge_weight = torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32)
edge_attr = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]], dtype=torch.float32)  # Edge features
relation = torch.rand(3, 4)  # Random relation features
input_features = torch.rand(4, 4)  # Node input features

# Call the function
output = generalized_rspmm(edge_index, edge_type, edge_weight, edge_attr, relation, input_features)
print("Output:", output)


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [7]:
import pickle


def inspect_cxtdict(cxtdict):
    print("Type:", type(cxtdict))
    print("Number of entries:", len(cxtdict))
    if isinstance(cxtdict, dict):
        print("Number of entries:", len(cxtdict))
        # Print a sample of the keys and values
        for i, key in enumerate(cxtdict.keys()):
            print(f"Key {i}: {key}, Value: {cxtdict[key]}")
            if i >= 9:  # Limit to first 10 entries
                break


def load_data(filename):
    try:
        with open(filename, "rb") as f:
            x= pickle.load(f)
    except:
        x = []
    return x
    
CXTDict = load_data('/itet-stor/trachsele/net_scratch/tl4rec/CARCA_Code_and_Data/Data/CXTDictSasRec_Games.dat')
inspect_cxtdict(CXTDict)
#print(CXTDict)
#print(CXTDict[(1,49583)])



Type: <class 'dict'>
Number of entries: 287107
Number of entries: 287107
Key 0: (352, 60), Value: [0.0, 0.5, 0.7419354838709677, 0.2857142857142857, 0.4767123287671233, 6.25]
Key 1: (1814, 647), Value: [0.0, 0.8333333333333334, 0.45161290322580644, 0.42857142857142855, 0.7863013698630137, 10.25]
Key 2: (1796, 619), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 3: (1504, 173), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 4: (1504, 364), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 5: (1504, 592), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 6: (1504, 785), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 7: (1504, 797), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.86027397

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
